# Bedrock Claude使用langchain agent搜索 YouTube 视频

## 安装依赖包

In [ ]:
!pip install -U langchain langchain_community langchain_core youtube_search --quiet

## 设置anthropic function 模版

In [ ]:
from langchain_community.utils.openai_functions import (
    FunctionDescription,
)
from langchain_core.tools import BaseTool


def format_tool_to_anthropic_function(tool: BaseTool) -> FunctionDescription:
    """Format tool into the Bedrock function API."""
    return f"""
<tool_description>
<tool_name>{tool.name}</tool_name>
<description>
{tool.description}
<parameters>
<parameter>
<name>__arg1</name>
<type>string</type>
<description>The input of the function</description>
</parameter>
</parameters>"""

## Claude 函数模板解析器AnthropicFunctionsAgentOutputParser

In [ ]:
import xml.etree.ElementTree as ET
from json import JSONDecodeError
from typing import List, Union
from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.outputs import ChatGeneration, Generation
from langchain.agents.agent import AgentOutputParser

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage
)


class AnthropicFunctionsAgentOutputParser(AgentOutputParser):

    @staticmethod
    def _extract_function_call(full_text):
        # Encontrar o início e o fim da string XML
        if "function_calls" not in full_text:
            return None

        start = full_text.find("<function_calls>")
        end = full_text.find("</function_calls>") + len("</function_calls>")

        # Extrair a string XML
        xml_string = full_text[start:end]

        # Análise da string XML
        root = ET.fromstring(xml_string)

        # Encontrar o nome da ferramenta
        tool_name = root.find(".//tool_name").text

        # Construir o dicionário de parâmetros
        parameters = {}
        for param in root.findall(".//parameters/*"):
            parameters[param.tag] = param.text

        # Construir o dicionário final com nome e parâmetros
        return {
            "name": tool_name,
            "arguments": parameters
        }

    @property
    def _type(self) -> str:
        return "openai-functions-agent"

    @staticmethod
    def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
        """Parse an AI message."""
        if not isinstance(message, AIMessage):
            raise TypeError(f"Expected an AI message got {type(message)}")

        function_call = AnthropicFunctionsAgentOutputParser._extract_function_call(message.content)

        if function_call and 'Final Answer:' not in message.content:
            function_name = function_call["name"]
            try:
                if len(function_call.keys()) == 0:
                    # OpenAI returns an empty string for functions containing no args
                    _tool_input = {}
                else:
                    # otherwise it returns a json object
                    _tool_input = function_call["arguments"]
            except JSONDecodeError:
                raise OutputParserException(
                    f"Could not parse tool input: {function_call} because "
                    f"the `arguments` is not valid JSON."
                )

            if "__arg1" in _tool_input.keys():
                tool_input = _tool_input["__arg1"]
            else:
                tool_input = _tool_input

            content_msg = f"responded: {message.content}\n" if message.content else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[message],
            )

        start = message.content.find("Final Answer:")

        return AgentFinish(
            return_values={"output": message.content[start:]}, log=str(message.content)
        )

    def parse_result(
            self, result: List[Generation], *, partial: bool = False
    ) -> Union[AgentAction, AgentFinish]:
        if not isinstance(result[0], ChatGeneration):
            raise ValueError("This output parser only works on ChatGeneration output")
        message = result[0].message
        return self._parse_ai_message(message)

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        raise ValueError("Can only parse messages")


## 加载Claude大模型并且设置prompt 模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat


prompt_template = """
You are a helfull assistant, your task is help the user achieve is goal.
In this environment you have access to a set of tools you can use to answer the user's question.
Use them in a smart way to fullfill our objetives. Always consider the history of previous steps to ensure a continuos progress towards the objetive.
When you have the answer for the user, always answer it in this forma bellow:
Final Answer: answer to user


You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>


User:
{input}

<history>
{agent_scratchpad}
</history>
"""

llm = BedrockChat(model_id="anthropic.claude-v2:1")

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template),
    ]
)

## 构建 Agent

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.youtube.search import YouTubeSearchTool
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)

agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm
        | AnthropicFunctionsAgentOutputParser()
)

tools = [YouTubeSearchTool()]
tools_string = f"<tools>{''.join([format_tool_to_anthropic_function(t) for t in tools])}</tools>"

## 执行 Agent

In [ ]:
from langchain.agents.agent import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "找出 5 个老高与小茉的视频",
                       "tools_string": f"<tools>{tools_string}</tools>"})